# Analyzing nanopore sequencing data for AMBIC

## Set up
Necessary softwares should be installed already in here.  
They are described in README.md as well as environment.yml.
Each step will have the actuall command as well as the snakemake command to do the same thing.
Check snakemake_config.yml and make sure that you've edited it to appropriate values for your application.

In [30]:
# first set working directory
workdir=~/Data
codedir=$(readlink -f $PWD/../) # this script is inside the repo/analysis, so go up a path
cd $codedir
reference=/mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa
threads=12

# samples
sample=CHOK1IgG
sample=CHOK1host

## Alignment
### minimap2
First make index for minimap2
```
minimap2 -x map-ont -d [path/to/index] [path/to/reference.fa]
```

* -x map-ont : presets for mapping ONT reads
* -d [index] : path to output index file
* input reference fasta path

In [17]:
mmi=${reference%.fa}.map_ont.mmi
com="minimap2 -x map-ont -d $mmi $reference"
echo $com
if [ ! -e $mmi ]; then
    eval $com
fi

minimap2 -x map-ont -d /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa


In [16]:
ls -lh $mmi

-rw-r--r-- 1 jupyter-isac jupyter-isac 5.6G Oct 23 11:59 /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi


Then align
```
minimap2 --MD -L -t [threads] -a [path/to/index] [query] |\
```
* --MD : include MD tag (used in sniffles)
* -t : cores
* -a : index generated in previous step
* query fastq file

Followed by filter and sort
```
samtools view -q 20 -b - |\
samtools sort -T [tmp] -o [output bam]
```
* -q 20 : filter out reads having mapping quality less than 20
* -b : output bam format
* -T : temporary files prefix
* -o : output bam file

In [20]:
[ -e $workdir/bam ]||mkdir $workdir/bam
# IgG
sample=CHOK1IgG
fq=$workdir/reads/$sample.fastq.gz
outpre=$workdir/bam/$sample.minimap2
output=$outpre.sorted.bam
com="minimap2 --MD -L -t $threads -a $mmi ${fq} |\
    samtools view -q 20 -b - |\
    samtools sort -T $outpre.sorting -o ${output} &&\
    samtools index ${output}"
echo $com
eval $com

# host
sample=CHOK1host
fq=$workdir/reads/$sample.fastq.gz
outpre=$workdir/bam/$sample.minimap2
output=$outpre.sorted.bam
com="minimap2 --MD -L -t $threads -a $mmi ${fq} |\
    samtools view -q 20 -b - |\
    samtools sort -T $outpre.sorting -o ${output} &&\
    samtools index ${output}"
echo $com
eval $com

minimap2 --MD -L -t 12 -a /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi /home/jupyter-isac/Data_tmp/reads/CHOK1IgG.fastq.gz | samtools view -q 20 -b - | samtools sort -T /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorting -o /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.bam && samtools index /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.bam
[M::main::6.284*1.00] loaded/built the index for 1831 target sequence(s)
[M::mm_mapopt_update::7.929*1.00] mid_occ = 490
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1831
[M::mm_idx_stat::8.796*1.00] distinct minimizers: 92926228 (39.14% are singletons); average occurrences: 4.825; average spacing: 5.284
[M::worker_pipeline::12.629*2.81] mapped 3258 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 --MD -L -t 12 -a /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi /home/jupyter-isac/Data_tmp/reads/CHOK1IgG.fastq.gz
[M::main] Real time: 12

In [22]:
ls -lh $workdir/bam

total 44M
-rw-r--r-- 1 jupyter-isac jupyter-isac  616 Oct 28 01:19 CHOK1IgG.minimap2.align.log
-rw-r--r-- 1 jupyter-isac jupyter-isac  36M Oct 31 02:43 CHOK1IgG.minimap2.sorted.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac 692K Oct 31 02:43 CHOK1IgG.minimap2.sorted.bam.bai
-rw-r--r-- 1 jupyter-isac jupyter-isac  614 Oct 28 01:19 CHOK1host.minimap2.align.log
-rw-r--r-- 1 jupyter-isac jupyter-isac 7.5M Oct 31 02:43 CHOK1host.minimap2.sorted.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac  51K Oct 31 02:43 CHOK1host.minimap2.sorted.bam.bai


snakemake commands : 

In [83]:
rm $workdir/bam/*

output=bam/CHOK1IgG.minimap2.sorted.bam
snakemake --cores $threads -p $output

output=bam/CHOK1host.minimap2.sorted.bam
snakemake --cores $threads -p $output

rm: cannot remove '/home/jupyter-isac/Data_tmp/bam/*': No such file or directory
Building DAG of jobs...
Nothing to be done.
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T051629.439714.snakemake.log
Building DAG of jobs...
MissingInputException in line 38 of /home/jupyter-isac/ambic-epigenome-dev/snakemake/nanopore_data_parse.smk:
Missing input files for rule minimap2_align:
reads/CHOK1host.fastq.gz


: 1

In [39]:
ls -lh $workdir/bam

total 44M
-rw-r--r-- 1 jupyter-isac jupyter-isac  615 Oct 31 02:54 CHOK1IgG.minimap2.align.log
-rw-r--r-- 1 jupyter-isac jupyter-isac  36M Oct 31 02:54 CHOK1IgG.minimap2.sorted.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac 692K Oct 31 02:54 CHOK1IgG.minimap2.sorted.bam.bai
-rw-r--r-- 1 jupyter-isac jupyter-isac  615 Oct 31 02:54 CHOK1host.minimap2.align.log
-rw-r--r-- 1 jupyter-isac jupyter-isac 7.5M Oct 31 02:54 CHOK1host.minimap2.sorted.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac  51K Oct 31 02:54 CHOK1host.minimap2.sorted.bam.bai


## Variant detection
Sniffles is used to call structural variations.  
Only the alignment bam file is necessary for SV calling.   
This is split into four parts : 
1. Separately call SVs
2. Merge all SVs
3. Force-call SVs in candidates provided by the merged calls in (2)  
4. Merge the resulting force-called SVs

### first round of calls
To call SVs : 
```
sniffles -m [path/to/bam] -v [path/to/vcf] -t [threads] \
    --tmp_file [path/to/tmp] -s 2 
```
* -m : input bam file
* -v : output vcf file
* -t : threads
* --tmp_file : temporary file path
* -s 2 : report any SVs that have at least 2 reads that support the SV

Because sniffles output is not sorted, we need to sort the resulting VCF file :
```
bcftools sort -o [path/to/output.vcf] -T [tmp/dir] [path/to/input.vcf]
```
* -o : sorted output path
* -T : temporary file path
* input vcf from SV calls

In [43]:
svdir=$workdir/sv
[ -e $svdir ]||mkdir $svdir

sample=CHOK1host
bam=$workdir/bam/$sample.minimap2.sorted.bam
vcftmp=$svdir/$sample.minimap2.sniffles.unsorted.vcf
vcf=$svdir/$sample.minimap2.sniffles.sorted.vcf

com="sniffles -m $bam -v $vcftmp -t $threads \
    --tmp_file $vcf.tmp \
    -s 2  &&\
    bcftools sort -o $vcf -T $svdir $vcftmp"
echo $com
eval $com
rm $vcftmp

sample=CHOK1IgG
bam=$workdir/bam/$sample.minimap2.sorted.bam
vcftmp=$svdir/$sample.minimap2.sniffles.unsorted.vcf
vcf=$svdir/$sample.minimap2.sniffles.sorted.vcf

com="sniffles -m $bam -v $vcftmp -t $threads \
    --tmp_file $vcf.tmp \
    -s 2  &&\
    bcftools sort -o $vcf -T $svdir $vcftmp"
echo $com
eval $com
rm $vcftmp

sniffles -m /home/jupyter-isac/Data_tmp/bam/CHOK1host.minimap2.sorted.bam -v /home/jupyter-isac/Data_tmp/sv/CHOK1host.minimap2.sniffles.unsorted.vcf -t 12 --tmp_file /home/jupyter-isac/Data_tmp/sv/CHOK1host.minimap2.sniffles.sorted.vcf.tmp -s 2 && bcftools sort -o /home/jupyter-isac/Data_tmp/sv/CHOK1host.minimap2.sniffles.sorted.vcf -T /home/jupyter-isac/Data_tmp/sv /home/jupyter-isac/Data_tmp/sv/CHOK1host.minimap2.sniffles.unsorted.vcf
Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.0745178
	Avg INS ratio: 0.0408241
Start parsing... RAZU01000102.1
	Switch Chr RAZU01000102.1
	Switch Chr RAZU01000002.1
	Switch Chr 4_0cdhfr_vrc01wtg1m3_dgv
Finalizing  ..
Writing to /home/jupyter-isac/Data_tmp/sv
Merging 1 temporary files
Cleaning
Done
sniffles -m /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.bam -v /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf -t 12 --tmp_file /home/jupyter-isac/Data

In [47]:
rm $workdir/sv/*
vcf=sv/CHOK1host.minimap2.sniffles.sorted.vcf
snakemake -j 12 -p $vcf
vcf=sv/CHOK1IgG.minimap2.sniffles.sorted.vcf
snakemake -j 12 -p $vcf

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	sniffles_sv
	1	sort_vcf
	2

[Thu Oct 31 03:11:41 2019]
rule sniffles_sv:
    input: bam/CHOK1host.minimap2.sorted.bam
    output: sv/CHOK1host.minimap2.sniffles.unsorted.vcf
    log: sv/CHOK1host.minimap2.sniffles.log
    jobid: 1
    wildcards: sample=CHOK1host.minimap2
    threads: 12

sniffles -m bam/CHOK1host.minimap2.sorted.bam -v sv/CHOK1host.minimap2.sniffles.unsorted.vcf -t 12 --tmp_file sv/CHOK1host.minimap2.sniffles.tmp -s 2  &> sv/CHOK1host.minimap2.sniffles.log
[Thu Oct 31 03:11:41 2019]
Finished job 1.
1 of 2 steps (50%) done

[Thu Oct 31 03:11:41 2019]
rule sort_vcf:
    input: sv/CHOK1host.minimap2.sniffles.unsorted.vcf
    output: sv/CHOK1host.minimap2.sniffles.sorted.vcf
    jobid: 0
    wildcards: sample=CHOK1host.minimap2.sniffles

bcftools sort -o sv/CHOK1host.minimap2.sniffles.sorted.vcf -T sv sv/CHOK1host.minimap2.sniffles.u

In [48]:
ls -lh $workdir/sv

total 228K
-rw-r--r-- 1 jupyter-isac jupyter-isac  311 Oct 31 03:11 CHOK1IgG.minimap2.sniffles.log
-rw-r--r-- 1 jupyter-isac jupyter-isac 125K Oct 31 03:11 CHOK1IgG.minimap2.sniffles.sorted.vcf
-rw-r--r-- 1 jupyter-isac jupyter-isac  290 Oct 31 03:11 CHOK1host.minimap2.sniffles.log
-rw-r--r-- 1 jupyter-isac jupyter-isac  90K Oct 31 03:11 CHOK1host.minimap2.sniffles.sorted.vcf


### merge calls
Using SURVIVOR to merge calls.
First save the list of vcfs in a file, separated by newlines :
```
echo [vcfs] | tr " " "\n" > [filelist.txt]
```
Then use feed this list to SURVIVOR :
```
SURVIVOR merge [filelist.txt] 1000 1 1 -1 -1 -1 [out.vcf]
```

In [49]:
vcfs=$(find $workdir/sv -name "*sniffles.sorted.vcf")
raw=$workdir/sv/raw_calls.txt
merged=$workdir/sv/SURVIVOR_merged_1kbpdist_typesave.vcf
echo $vcfs | tr " " "\n" > $raw

SURVIVOR merge $raw 1000 1 1 -1 -1 -1 $merged

merging entries: 116
merging entries: 33


In [51]:
rm $workdir/sv/*
merged=sv/SURVIVOR_merged_1kbpdist_typesave.vcf
snakemake -j 12 -p $merged

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	merge_sv_before_forcecall
	2	sniffles_sv
	2	sort_vcf
	5

[Thu Oct 31 03:15:03 2019]
rule sniffles_sv:
    input: bam/CHOK1IgG.minimap2.sorted.bam
    output: sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf
    log: sv/CHOK1IgG.minimap2.sniffles.log
    jobid: 3
    wildcards: sample=CHOK1IgG.minimap2
    threads: 12

sniffles -m bam/CHOK1IgG.minimap2.sorted.bam -v sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf -t 12 --tmp_file sv/CHOK1IgG.minimap2.sniffles.tmp -s 2  &> sv/CHOK1IgG.minimap2.sniffles.log
[Thu Oct 31 03:15:06 2019]
Finished job 3.
1 of 5 steps (20%) done

[Thu Oct 31 03:15:06 2019]
rule sort_vcf:
    input: sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf
    output: sv/CHOK1IgG.minimap2.sniffles.sorted.vcf
    jobid: 1
    wildcards: sample=CHOK1IgG.minimap2.sniffles

bcftools sort -o sv/CHOK1IgG.minimap2.sniffles.sorted.vcf -T sv sv/CHOK1IgG.min

### force calls for SVs in merged calls
Redo SV calling on individual samples.  
paramters are mostly the same, but this time we provide the candidate VCF.  
```
sniffles -m [path/to/bam] -v [path/to/out.vcf] -t [threads] \
    --Ivcf [merged.vcf] --tmp_file [path/to/tmp] \
     -s 2 -n -1
```
* --Ivcf : candidate vcf to force-call SVs in these
* -n -1 : report query names of all reads that support the SV


In [52]:
sample=CHOK1host
bam=$workdir/bam/$sample.minimap2.sorted.bam
merged=$workdir/sv/SURVIVOR_merged_1kbpdist_typesave.vcf
vcftmp=$workdir/sv/$sample.SURVIVOR.unsorted.vcf
vcf=$workdir/sv/$sample.SURVIVOR.sorted.vcf
com="sniffles -m $bam -v $vcf -t $threads \
    --Ivcf $merged --tmp_file \
    $workdir/sv/$samp.SURVIVORsniffles.tmp \
    -s 2 -n -1"
echo $com
eval $com

sample=CHOK1IgG
bam=$workdir/bam/$sample.minimap2.sorted.bam
merged=$workdir/sv/SURVIVOR_merged_1kbpdist_typesave.vcf
vcftmp=$workdir/sv/$sample.SURVIVOR.unsorted.vcf
vcf=$workdir/sv/$sample.SURVIVOR.sorted.vcf
com="sniffles -m $bam -v $vcf -t $threads \
    --Ivcf $merged --tmp_file \
    $workdir/sv/$samp.SURVIVORsniffles.tmp \
    -s 2 -n -1"
echo $com
eval $com

sniffles -m /home/jupyter-isac/Data_tmp/bam/CHOK1host.minimap2.sorted.bam -v /home/jupyter-isac/Data_tmp/sv/CHOK1host.SURVIVOR.sorted.vcf -t 12 --Ivcf /home/jupyter-isac/Data_tmp/sv/SURVIVOR_merged_1kbpdist_typesave.vcf --tmp_file /home/jupyter-isac/Data_tmp/sv/.SURVIVORsniffles.tmp -s 2 -n -1
Automatically enabling genotype mode
Force calling SVs
Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.0745178
	Avg INS ratio: 0.0408241
Construct Tree...
		109 SVs found in input.
	Invalid types found skipping 0 entries.
Start parsing: Chr RAZU01000074.1
	Switch Chr RAZU01000102.1
	Switch Chr RAZU01000002.1
	Switch Chr 4_0cdhfr_vrc01wtg1m3_dgv
Finalizing  ..
Start genotype calling:
	Construct tree
	Update reference alleles
		Scanning CHR RAZU01000102.1
		Scanning CHR RAZU01000002.1
		Scanning CHR 4_0cdhfr_vrc01wtg1m3_dgv
	Writing SV calls
	Cleaning tmp files
sniffles -m /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.

### Final merging
Finally merge the force-called SVs using the same commands with slight change in the SURVIVOR options
```
SURVIVOR merge [filelist.txt] 1000 -1 1 -1 -1 -1 [out.vcf]
```


In [ ]:
raw=$workdir/sv/SURVIVORsniffles_calls.txt
vcfs=$(find $workdir/sv -name "*SURVIVOR.sorted.vcf")
merged=$workdir/sv/merged_final_SURVIVOR_1kbpdist_typesave.vcf

echo $vcfs | tr " " "\n" > $raw
SURVIVOR merge $raw 1000 -1 1 -1 -1 -1 $merged

### Run snakemake to automate all of the SV calling process
simply give snakemake the name of the resulting vcf to run all necessary commands to generate the file

In [54]:
rm $workdir/sv/*
vcf_final=sv/merged_final_SURVIVOR_1kbpdist_typesave.vcf
com="snakemake -j 12 $vcf_final"
echo $com
eval $com

snakemake -j 12 sv/merged_final_SURVIVOR_1kbpdist_typesave.vcf
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	merge_sv_before_forcecall
	1	merge_sv_final
	2	sniffles_sv
	2	sniffles_sv_forcecall
	4	sort_vcf
	10

[Thu Oct 31 03:23:04 2019]
rule sniffles_sv:
    input: bam/CHOK1IgG.minimap2.sorted.bam
    output: sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf
    log: sv/CHOK1IgG.minimap2.sniffles.log
    jobid: 11
    wildcards: sample=CHOK1IgG.minimap2
    threads: 12

[Thu Oct 31 03:23:07 2019]
Finished job 11.
1 of 10 steps (10%) done

[Thu Oct 31 03:23:07 2019]
rule sort_vcf:
    input: sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf
    output: sv/CHOK1IgG.minimap2.sniffles.sorted.vcf
    jobid: 9
    wildcards: sample=CHOK1IgG.minimap2.sniffles

Writing to sv
Merging 1 temporary files
Cleaning
Done
Removing temporary output file sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf.
[Thu Oct 31 03:23:07 2019

### Output file
The output of this is a merged vcf,  
where each row represents an SV,  
and the last N columns correspond to the SV calling result for each of the N samples.

In [81]:
grep "#CHROM" $workdir/$vcf_final | tail
grep "=TRA" $workdir/$vcf_final | head -n1

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	bam/CHOK1IgG.minimap2.sorted.bam	bam/CHOK1host.minimap2.sorted.bam
RAZU01000002.1	6220834	9	N	N[4_0cdhfr_vrc01wtg1m3_dgv:11910[	.	PASS	SUPP=1;SUPP_VEC=10;SVLEN=0;SVTYPE=TRA;SVMETHOD=SURVIVOR1.0.6;CHR2=4_0cdhfr_vrc01wtg1m3_dgv;END=11910;CIPOS=0,0;CIEND=0,0;STRANDS=++	GT:PSV:LN:DR:ST:QV:TY:ID:RAL:AAL:CO	1/1:NA:6208924:4,19:++:.:TRA:9_0:NA:NA:RAZU01000002.1_6220834-4_0cdhfr_vrc01wtg1m3_dgv_11910	0/0:NA:6208924:2,0:++:.:TRA:9:NA:NA:RAZU01000002.1_6220834-4_0cdhfr_vrc01wtg1m3_dgv_11910


For visualization of these SVs, genome ribbon is a good tool (http://genomeribbon.com/)
![genome_ribbon](../img/CHOK1IgG_ribbon.PNG)

## Metylation calling
We use nanopolish for methylation calling 
### nanopolish index
First step is to index the reads.  
This step scans through the reads and fast5 files to fiind the association between the reads and fast5 files.  
```
nanopolish index -d [path/to/fast5s] [ -s [path/to/sequencing_summary.txt] ] [path/to/reads.fastq(a)]
```
* -d : directory of fast5s ; nanopolish requires fast5 files to be around and uncompressed
* -s (optional) : Because the sequencing_summary.txt file has the fast5 names for each read, users can provide the summary file to dramatically speed up the indexing process.
* input fastq/a : input fastq/a file of the basecalled reads

In [60]:
rdir=$workdir/reads/CHOK1IgG
fq=$workdir/reads/CHOK1IgG.fastq.gz
nanopolish index -d $rdir $fq

rdir=$workdir/reads/CHOK1host
fq=$workdir/reads/CHOK1host.fastq.gz
nanopolish index -d $rdir $fq

[readdb] indexing /home/jupyter-isac/Data_tmp/reads/CHOK1IgG
[readdb] num reads: 3258, num reads with path to fast5: 3258
[readdb] indexing /home/jupyter-isac/Data_tmp/reads/CHOK1host
[readdb] num reads: 878, num reads with path to fast5: 878


In [66]:
rm $workdir/reads/*index*
output=reads/CHOK1IgG.fastq.gz.index.readdb
snakemake -p $output
output=reads/CHOK1host.fastq.gz.index.readdb
snakemake -p $output

rm: cannot remove '/home/jupyter-isac/Data_tmp/reads/*index*': No such file or directory
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	nanopolish_index
	1

[Thu Oct 31 03:38:08 2019]
rule nanopolish_index:
    input: reads/CHOK1IgG.fastq.gz
    output: reads/CHOK1IgG.fastq.gz.index.readdb
    jobid: 0
    wildcards: pre=reads/CHOK1IgG

nanopolish index --verbose -d reads/CHOK1IgG reads/CHOK1IgG.fastq.gz &> reads/CHOK1IgG.index.log
[Thu Oct 31 03:38:09 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T033807.886540.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	nanopolish_index
	1

[Thu Oct 31 03:38:10 2019]
rule nanopolish_index:
    input: reads/CHOK1host.fastq.gz
    output: reads/CHOK1host.fastq.gz.index.read

In [67]:
ls -lh $workdir/reads

total 41M
drwxr-xr-x 2 jupyter-isac jupyter-isac 480K Oct 28 01:18 CHOK1IgG
-rw-r--r-- 1 jupyter-isac jupyter-isac  25M Oct 28 01:18 CHOK1IgG.fastq.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 7.3M Oct 31 03:38 CHOK1IgG.fastq.gz.index
-rw-r--r-- 1 jupyter-isac jupyter-isac 193K Oct 31 03:38 CHOK1IgG.fastq.gz.index.fai
-rw-r--r-- 1 jupyter-isac jupyter-isac 6.4K Oct 31 03:38 CHOK1IgG.fastq.gz.index.gzi
-rw-r--r-- 1 jupyter-isac jupyter-isac 471K Oct 31 03:38 CHOK1IgG.fastq.gz.index.readdb
-rw-r--r-- 1 jupyter-isac jupyter-isac   94 Oct 31 03:38 CHOK1IgG.index.log
drwxr-xr-x 2 jupyter-isac jupyter-isac 124K Oct 28 01:18 CHOK1host
-rw-r--r-- 1 jupyter-isac jupyter-isac 5.9M Oct 28 01:18 CHOK1host.fastq.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.8M Oct 31 03:38 CHOK1host.fastq.gz.index
-rw-r--r-- 1 jupyter-isac jupyter-isac  52K Oct 31 03:38 CHOK1host.fastq.gz.index.fai
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.6K Oct 31 03:38 CHOK1host.fastq.gz.index.gzi
-rw-r--r-- 1 jupyter-isac jupyter-isa

### call methylation
Once indexing is done, methylation can be called :
```
nanopolish call-methylation -t [threads] -q cpg \
    -g [path/to/reference] -r [path/to/fastq] -b [path/to/bam] |\
    gzip > [path/to/out.gz]
```
* -q cpg : specifying that CpG methylation is being called
* -g : path for reference fasta
* -r : path to the fastq/a file that has been index by nanopolish
* -b : path to the bam alignments file

In [58]:
outdir=$workdir/methylation
[ -e $outdir ]||mkdir $outdir

sample=CHOK1IgG
fastq=$workdir/reads/$sample.fastq.gz
bam=$workdir/bam/$sample.minimap2.sorted.bam
output=$outdir/$sample.cpg.meth.tsv.gz

nanopolish call-methylation -v -t ${threads} -q cpg \
    -g $reference -r $fastq -b $bam |\
    gzip > $output
    
sample=CHOK1host
fastq=$workdir/reads/$sample.fastq.gz
bam=$workdir/bam/$sample.minimap2.sorted.bam
output=$outdir/$sample.cpg.meth.tsv.gz

nanopolish call-methylation -v -t ${threads} -q cpg \
    -g $reference -r $fastq -b $bam |\
    gzip > $output

[post-run summary] total reads: 6388, unparseable: 0, qc fail: 1, could not calibrate: 3, no alignment: 867, bad fast5: 0


In [68]:
#rm $workdir/methylation/*
sample=CHOK1IgG
output=methylation/$sample.cpg.meth.tsv.gz
snakemake -j $threads -p $output

sample=CHOK1host
output=methylation/$sample.cpg.meth.tsv.gz
snakemake -j $threads -p $output

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	call_cpg
	1

[Thu Oct 31 04:12:58 2019]
rule call_cpg:
    input: reads/CHOK1IgG.fastq.gz, bam/CHOK1IgG.minimap2.sorted.bam, reads/CHOK1IgG.fastq.gz.index.readdb
    output: methylation/CHOK1IgG.cpg.meth.tsv.gz
    jobid: 0
    wildcards: sample=CHOK1IgG
    threads: 12

nanopolish call-methylation -v -t 12 -q cpg -g /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa -r reads/CHOK1IgG.fastq.gz -b bam/CHOK1IgG.minimap2.sorted.bam | gzip > methylation/CHOK1IgG.cpg.meth.tsv.gz
[post-run summary] total reads: 8118, unparseable: 0, qc fail: 1, could not calibrate: 4, no alignment: 977, bad fast5: 0
[Thu Oct 31 04:14:35 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T041258.024053.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12

In [69]:
ls -lh $workdir/methylation

total 19M
-rw-r--r-- 1 jupyter-isac jupyter-isac 9.6M Oct 28 03:12 CHOK1IgG.cpg.meth.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac 640K Oct 28 03:12 CHOK1IgG.cpg.meth.bam.bai
-rw-r--r-- 1 jupyter-isac jupyter-isac 986K Oct 28 02:16 CHOK1IgG.cpg.meth.bed.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 5.5K Oct 28 02:16 CHOK1IgG.cpg.meth.bed.gz.tbi
-rw-r--r-- 1 jupyter-isac jupyter-isac 3.8M Oct 31 04:14 CHOK1IgG.cpg.meth.tsv.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac   18 Oct 28 02:26 CHOK1IgG.cpg.mfreq.log
-rw-r--r-- 1 jupyter-isac jupyter-isac  22K Oct 28 02:26 CHOK1IgG.cpg.mfreq.txt.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.4K Oct 28 02:26 CHOK1IgG.cpg.mfreq.txt.gz.tbi
-rw-r--r-- 1 jupyter-isac jupyter-isac 2.5M Oct 28 03:12 CHOK1host.cpg.meth.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac  51K Oct 28 03:12 CHOK1host.cpg.meth.bam.bai
-rw-r--r-- 1 jupyter-isac jupyter-isac 277K Oct 28 02:17 CHOK1host.cpg.meth.bed.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac  458 Oct 28 02:17 CHOK1host.cpg.meth.bed.g

### make methylation bed files
Methylation output from nanopolish is one call per line and not genomically sorted.  
This makes the output large and hard to query calls in certain regions.  
So I convert them to a bed-style format which can be indexed using tabix for much faster access. 
```
python $codedir/nanopore-methylation-utilities/mtsv2bedGraph.py -g [path/to/reference.fa] -i [path/to/nanopolish.tsv]  |
```
* -g : path to reference genome
* -i : nanopolish methylation output  

Then I sort and bgzip the output, followed by tabix indexing
```
sort -T tmp -k1,1 -k2,2n | 
bgzip > [output.bed.gz] && 
tabix -p bed [output.bed.gz]
```


In [71]:
sample=CHOK1IgG
input=$workdir/methylation/$sample.cpg.meth.tsv.gz
output=$workdir/methylation/$sample.cpg.meth.bed.gz

python $codedir/nanopore-methylation-utilities/mtsv2bedGraph.py -g $reference -i $input |\
    sort -k1,1 -k2,2n |\
    bgzip > $output &&\
    tabix -p bed $output
    
sample=CHOK1host
input=$workdir/methylation/$sample.cpg.meth.tsv.gz
output=$workdir/methylation/$sample.cpg.meth.bed.gz

python $codedir/nanopore-methylation-utilities/mtsv2bedGraph.py -g $reference -i $input |\
    sort -k1,1 -k2,2n |\
    bgzip > $output &&\
    tabix -p bed $output

In [73]:
rm $workdir/methylation/*meth.bed.gz
sample=CHOK1IgG
mbed=methylation/$sample.cpg.meth.bed.gz
snakemake -p $mbed
sample=CHOK1host
mbed=methylation/$sample.cpg.meth.bed.gz
snakemake -p $mbed

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	mtsv_to_mbed
	1

[Thu Oct 31 04:25:28 2019]
rule mtsv_to_mbed:
    input: methylation/CHOK1IgG.cpg.meth.tsv.gz, /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa
    output: methylation/CHOK1IgG.cpg.meth.bed.gz
    jobid: 0
    wildcards: sample=CHOK1IgG

python ~/ambic-epigenome-dev/nanopore-methylation-utilities/mtsv2bedGraph.py -g /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa -i methylation/CHOK1IgG.cpg.meth.tsv.gz | sort -T tmp -k1,1 -k2,2n | bgzip > methylation/CHOK1IgG.cpg.meth.bed.gz && tabix -p bed methylation/CHOK1IgG.cpg.meth.bed.gz
[Thu Oct 31 04:25:42 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T042528.461312.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads wi

### make methylation frequency files
We can then parse the methylation bed files to generate methylation frequencies.  
```
python -u $codedir/nanopore-methylation-utilities/parseMethylbed.py frequency -i [path/to/input.bed.gz] -m cpg |
```
The output is already sorted so no additional sorting is necessary, and this can also be bgzipped and tabixed for easy access.  

```
bgzip > [path/to/output.gz] tabix -b 2 -e 2 [path/to/output.gz]
```

In [74]:
sample=CHOK1IgG
input=$workdir/methylation/$sample.cpg.meth.bed.gz
output=$workdir/methylation/$sample.cpg.mfreq.txt.gz
python -u $codedir/nanopore-methylation-utilities/parseMethylbed.py frequency -i $input -m cpg |\
    bgzip > $output && tabix -b 2 -e 2 $output
    
sample=CHOK1host
input=$workdir/methylation/$sample.cpg.meth.bed.gz
output=$workdir/methylation/$sample.cpg.mfreq.txt.gz
python -u $codedir/nanopore-methylation-utilities/parseMethylbed.py frequency -i $input -m cpg |\
    bgzip > $output && tabix -b 2 -e 2 $output

In [76]:
rm $workdir/methylation/*mfreq.txt.gz
sample=CHOK1IgG
mfreq=methylation/$sample.cpg.mfreq.txt.gz
snakemake $mfreq

sample=CHOK1host
mfreq=methylation/$sample.cpg.mfreq.txt.gz
snakemake $mfreq

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	mbed_to_mfreq
	1

[Thu Oct 31 04:28:32 2019]
rule mbed_to_mfreq:
    input: methylation/CHOK1IgG.cpg.meth.bed.gz
    output: methylation/CHOK1IgG.cpg.mfreq.txt.gz
    log: methylation/CHOK1IgG.cpg.mfreq.log
    jobid: 0
    wildcards: sample=CHOK1IgG, mod=cpg

[Thu Oct 31 04:28:33 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T042832.126751.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	mbed_to_mfreq
	1

[Thu Oct 31 04:28:34 2019]
rule mbed_to_mfreq:
    input: methylation/CHOK1host.cpg.meth.bed.gz
    output: methylation/CHOK1host.cpg.mfreq.txt.gz
    log: methylation/CHOK1host.cpg.mfreq.log
    jobid: 0
    wildcards: sample=CHOK1host, mod=cpg

[Thu Oct 31 04:28

The resulting methylation frequency file is the same format as bismark methylation report, so the NGS BS-seq analysis methods can be used.  
One example is to load onto R and plot a region of interest.

![methylation](../img/CHOK1IgG_methylation.PNG)

### make methylation bam
Another thing you can do is mark the bam files with methylation and view methylation on IGV using IGV's bisulfite mode.  
```
python $codedir/nanopore-methylation-utilities/convert_bam_for_methylation.py -t [threads] -c [path/to/cpg.bed.gz] -b [path/to/bam] |\
    samtools sort -o [path/to/out.bam] && samtools index [path/to/out.bam]
```
* -c : path to cpg.meth.bed.gz 
* -b : path to bam file
Once the methylation encoded bam files are generated, they can be exported to IGV to see methylation on individual reads

In [79]:
sample=CHOK1IgG
methbed=$workdir/methylation/$sample.cpg.meth.bed.gz
bam=$workdir/bam/$sample.minimap2.sorted.bam
output=$workdir/methylation/$sample.cpg.meth.bam
python $codedir/nanopore-methylation-utilities/convert_bam_for_methylation.py -t $threads -c $methbed -b $bam |\
    samtools sort -o $output && samtools index $output
    
sample=CHOK1host
methbed=$workdir/methylation/$sample.cpg.meth.bed.gz
bam=$workdir/bam/$sample.minimap2.sorted.bam
output=$workdir/methylation/$sample.cpg.meth.bam
python $codedir/nanopore-methylation-utilities/convert_bam_for_methylation.py -t $threads -c $methbed -b $bam |\
    samtools sort -o $output && samtools index $output

In [80]:
rm $workdir/methylation/*bam*
sample=CHOK1host
outbam=methylation/$sample.cpg.meth.bam
snakemake -p -j $threads $outbam

sample=CHOK1IgG
outbam=methylation/$sample.cpg.meth.bam
snakemake -p -j $threads $outbam

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bam_methylation
	1

[Thu Oct 31 04:39:57 2019]
rule bam_methylation:
    input: methylation/CHOK1host.cpg.meth.bed.gz, bam/CHOK1host.minimap2.sorted.bam
    output: methylation/CHOK1host.cpg.meth.bam
    jobid: 0
    wildcards: sample=CHOK1host
    threads: 12

python ~/ambic-epigenome-dev/nanopore-methylation-utilities/convert_bam_for_methylation.py -t 12 -c methylation/CHOK1host.cpg.meth.bed.gz -b bam/CHOK1host.minimap2.sorted.bam | samtools sort -o methylation/CHOK1host.cpg.meth.bam && samtools index methylation/CHOK1host.cpg.meth.bam
[Thu Oct 31 04:40:00 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-31T043957.596446.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs


Visualizing on IGV with bisulfite viewing mode : 

Host :  
![methbam_host](../img/CHOK1host_methbam.PNG)

IgG :   
![methbam_igg](../img/CHOK1IgG_methbam.PNG)